In [2]:
import requests
import json
import numpy as np
import pandas as pd
import pprint
import math
import datetime
pd.options.display.max_columns = 999
# np.set_printoptions(suppress=True)
# pd.set_option('display.float_format', lambda x: '%.3f' % x)
shopify_data = pd.read_csv("pcsg_shopify_20180531.csv", low_memory=False, dtype={'Lineitem sku': 'str'})

def try_float(x):
    try:
        return str(format(float(x), '.0f'))
    except ValueError as e:
        return x

shopify_data['Name'] = shopify_data['Name'].apply(try_float)
sku_master = pd.read_excel("C:/Users/limzi/OneDrive/Forecasting & Reporting/PCSG Master List.xlsx", sheet_name='SKU List' )
bundle_master = pd.read_excel("C:/Users/limzi/OneDrive/Forecasting & Reporting/PCSG Master List.xlsx",
                              sheet_name='Bundles' ).rename(columns = {'Parent SKU': "parent_sku",
                                                                      'Parent Name': 'parent_name',
                                                                      'Quantity': 'child_quantity',
                                                                      'Child SKU': "child_sku",
                                                                      'Child Name': 'child_name'}).astype({'child_sku': 'str',
                                                                                                           'parent_sku': 'str'})

In [3]:
shopify_data_merge = pd.merge(shopify_data, bundle_master, left_on="Lineitem sku", right_on = 'parent_sku', how='left')
shopify_data_merge#[shopify_data_merge['Lineitem sku'] == '4707']

,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,Taxes,Total,Discount Code,Discount Amount,Shipping Method,Created at,Lineitem quantity,Lineitem name,Lineitem price,Lineitem compare at price,Lineitem sku,Lineitem requires shipping,Lineitem taxable,Lineitem fulfillment status,Billing Name,Billing Street,Billing Address1,Billing Address2,Billing Company,Billing City,Billing Zip,Billing Province,Billing Country,Billing Phone,Shipping Name,Shipping Street,Shipping Address1,Shipping Address2,Shipping Company,Shipping City,Shipping Zip,Shipping Province,Shipping Country,Shipping Phone,Notes,Note Attributes,Cancelled at,Payment Method,Payment Reference,Refunded Amount,Vendor,Outstanding Balance,Employee,Location,Device ID,Id,Tags,Risk Level,Source,Lineitem discount,Tax 1 Name,Tax 1 Value,Tax 2 Name,Tax 2 Value,Tax 3 Name,Tax 3 Value,Tax 4 Name,Tax 4 Value,Tax 5 Name,Tax 5 Value,Phone,parent_sku,parent_name,child_sku,child_name,child_quantity,Unit Price,Price,Component Retail,Bundle Retail,% Discount,Discount Unit Price
0,191570000000,jnbitalac@gmail.com,paid,2018-05-31 23:59:17 +0800,unfulfilled,NaN,no,SGD,42.0,0.0,0.0,42.0,NaN,0.0,Free Delivery (2 Working Days),2018-05-31 23:59:17 +0800,1,Clear Acne Regular Strength Travel Kit,29.0,NaN,4707,True,False,pending,Jane B,"3 Fusionopolis Way, 05-21 Symbiosis",3 Fusionopolis Way,05-21 Symbiosis,NaN,Singapore,'138633,NaN,SG,'+65 8246 2761,Jane B,"3 Fusionopolis Way, 05-21 Symbiosis",3 Fusionopolis Way,05-21 Symbiosis,NaN,Singapore,'138633,NaN,SG,'+65 8246 2761,NaN,NaN,NaN,Stripe,c3437700579393.1,0.0,Paula's Choice,0.0,NaN,NaN,NaN,4.944910e+11,NaN,Low,web,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4707,Clear Regular Strength Trial Kit,6006,30 ml,1.0,8.0,8.0,39.0,29.0,0.256410,5.948718
1,191570000000,jnbitalac@gmail.com,paid,2018-05-31 23:59:17 +0800,unfulfilled,NaN,no,SGD,42.0,0.0,0.0,42.0,NaN,0.0,Free Delivery (2 Working Days),2018-05-31 23:59:17 +0800,1,Clear Acne Regular Strength Travel Kit,29.0,NaN,4707,True,False,pending,Jane B,"3 Fusionopolis Way, 05-21 Symbiosis",3 Fusionopolis Way,05-21 Symbiosis,NaN,Singapore,'138633,NaN,SG,'+65 8246 2761,Jane B,"3 Fusionopolis Way, 05-21 Symbiosis",3 Fusionopolis Way,05-21 Symbiosis,NaN,Singapore,'138633,NaN,SG,'+65 8246 2761,NaN,NaN,NaN,Stripe,c3437700579393.1,0.0,Paula's Choice,0.0,NaN,NaN,NaN,4.944910e+11,NaN,Low,web,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4707,Clear Regular Strength Trial Kit,6107,14 ml,1.0,14.0,14.0,39.0,29.0,0.256410,10.410256
2,191570000000,jnbitalac@gmail.com,paid,2018-05-31 23:59:17 +0800,unfulfilled,NaN,no,SGD,42.0,0.0,0.0,42.0,NaN,0.0,Free Delivery (2 Working Days),2018-05-31 23:59:17 +0800,1,Clear Acne Regular Strength Travel Kit,29.0,NaN,4707,True,False,pending,Jane B,"3 Fusionopolis Way, 05-21 Symbiosis",3 Fusionopolis Way,05-21 Symbiosis,NaN,Singapore,'138633,NaN,SG,'+65 8246 2761,Jane B,"3 Fusionopolis Way, 05-21 Symbiosis",3 Fusionopolis Way,05-21 Symbiosis,NaN,Singapore,'138633,NaN,SG,'+65 8246 2761,NaN,NaN,NaN,Stripe,c3437700579393.1,0.0,Paula's Choice,0.0,NaN,NaN,NaN,4.944910e+11,NaN,Low,web,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4707,Clear Regular Strength Trial Kit,6207,30 ml,1.0,17.0,17.0,39.0,29.0,0.256410,12.641026
3,191570000000,jnbitalac@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-05-31 23:59:17 +0800,1,Skin Perfecting 2% BHA (Salicylic Acid) Liquid...,13.0,NaN,2016,True,False,pending,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Paula's Choice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,191570000000,isjanelle@gmail.com,paid,2018-05-31 23:54:52 +0800,unfulfilled,NaN,yes,SGD,68.0,0.0,0.0,68.0,NaN,0.0,Free Delivery (2 Working Days),2018-05-31 23:54:52 +0800,1,C15 Super Booster - 20 ml,68.0,NaN,7770,True,False,pending,Lee,Blk 186B Rivervale Drive #11-816,Blk 186B

In [4]:
shopify_data_merge['parent_sku'].fillna(shopify_data_merge['Lineitem sku'], inplace=True)
shopify_data_merge['child_sku'].fillna(shopify_data_merge['Lineitem sku'], inplace=True)
shopify_data_merge['child_quantity'].fillna(shopify_data_merge['Lineitem quantity'], inplace=True)
shopify_data_merge['Discount Unit Price'].fillna(shopify_data_merge['Lineitem price'], inplace=True)
# shopify_data_merge['Unit Price'].fillna(shopify_data_merge['Lineitem compare at price'], inplace=True)
# shopify_data_merge['total_quantity'].fillna(shopify_data_merge['Lineitem quantity'], inplace = True)
shopify_data_merge['child_subtotal_quantity'] =  shopify_data_merge['child_quantity'] * shopify_data_merge['Lineitem quantity']
shopify_data_merge['child_subtotal'] =  shopify_data_merge['child_subtotal_quantity'] * shopify_data_merge['Discount Unit Price']
shopify_data_final = shopify_data_merge.copy()


In [5]:
# shopify_data_final.to_excel('formatted_shopify_data.xlsx')
shopify_data_final

,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,Taxes,Total,Discount Code,Discount Amount,Shipping Method,Created at,Lineitem quantity,Lineitem name,Lineitem price,Lineitem compare at price,Lineitem sku,Lineitem requires shipping,Lineitem taxable,Lineitem fulfillment status,Billing Name,Billing Street,Billing Address1,Billing Address2,Billing Company,Billing City,Billing Zip,Billing Province,Billing Country,Billing Phone,Shipping Name,Shipping Street,Shipping Address1,Shipping Address2,Shipping Company,Shipping City,Shipping Zip,Shipping Province,Shipping Country,Shipping Phone,Notes,Note Attributes,Cancelled at,Payment Method,Payment Reference,Refunded Amount,Vendor,Outstanding Balance,Employee,Location,Device ID,Id,Tags,Risk Level,Source,Lineitem discount,Tax 1 Name,Tax 1 Value,Tax 2 Name,Tax 2 Value,Tax 3 Name,Tax 3 Value,Tax 4 Name,Tax 4 Value,Tax 5 Name,Tax 5 Value,Phone,parent_sku,parent_name,child_sku,child_name,child_quantity,Unit Price,Price,Component Retail,Bundle Retail,% Discount,Discount Unit Price,child_subtotal_quantity,child_subtotal
0,191570000000,jnbitalac@gmail.com,paid,2018-05-31 23:59:17 +0800,unfulfilled,NaN,no,SGD,42.0,0.0,0.0,42.0,NaN,0.0,Free Delivery (2 Working Days),2018-05-31 23:59:17 +0800,1,Clear Acne Regular Strength Travel Kit,29.0,NaN,4707,True,False,pending,Jane B,"3 Fusionopolis Way, 05-21 Symbiosis",3 Fusionopolis Way,05-21 Symbiosis,NaN,Singapore,'138633,NaN,SG,'+65 8246 2761,Jane B,"3 Fusionopolis Way, 05-21 Symbiosis",3 Fusionopolis Way,05-21 Symbiosis,NaN,Singapore,'138633,NaN,SG,'+65 8246 2761,NaN,NaN,NaN,Stripe,c3437700579393.1,0.0,Paula's Choice,0.0,NaN,NaN,NaN,4.944910e+11,NaN,Low,web,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4707,Clear Regular Strength Trial Kit,6006,30 ml,1.0,8.0,8.0,39.0,29.0,0.256410,5.948718,1.0,5.948718
1,191570000000,jnbitalac@gmail.com,paid,2018-05-31 23:59:17 +0800,unfulfilled,NaN,no,SGD,42.0,0.0,0.0,42.0,NaN,0.0,Free Delivery (2 Working Days),2018-05-31 23:59:17 +0800,1,Clear Acne Regular Strength Travel Kit,29.0,NaN,4707,True,False,pending,Jane B,"3 Fusionopolis Way, 05-21 Symbiosis",3 Fusionopolis Way,05-21 Symbiosis,NaN,Singapore,'138633,NaN,SG,'+65 8246 2761,Jane B,"3 Fusionopolis Way, 05-21 Symbiosis",3 Fusionopolis Way,05-21 Symbiosis,NaN,Singapore,'138633,NaN,SG,'+65 8246 2761,NaN,NaN,NaN,Stripe,c3437700579393.1,0.0,Paula's Choice,0.0,NaN,NaN,NaN,4.944910e+11,NaN,Low,web,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4707,Clear Regular Strength Trial Kit,6107,14 ml,1.0,14.0,14.0,39.0,29.0,0.256410,10.410256,1.0,10.410256
2,191570000000,jnbitalac@gmail.com,paid,2018-05-31 23:59:17 +0800,unfulfilled,NaN,no,SGD,42.0,0.0,0.0,42.0,NaN,0.0,Free Delivery (2 Working Days),2018-05-31 23:59:17 +0800,1,Clear Acne Regular Strength Travel Kit,29.0,NaN,4707,True,False,pending,Jane B,"3 Fusionopolis Way, 05-21 Symbiosis",3 Fusionopolis Way,05-21 Symbiosis,NaN,Singapore,'138633,NaN,SG,'+65 8246 2761,Jane B,"3 Fusionopolis Way, 05-21 Symbiosis",3 Fusionopolis Way,05-21 Symbiosis,NaN,Singapore,'138633,NaN,SG,'+65 8246 2761,NaN,NaN,NaN,Stripe,c3437700579393.1,0.0,Paula's Choice,0.0,NaN,NaN,NaN,4.944910e+11,NaN,Low,web,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4707,Clear Regular Strength Trial Kit,6207,30 ml,1.0,17.0,17.0,39.0,29.0,0.256410,12.641026,1.0,12.641026
3,191570000000,jnbitalac@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-05-31 23:59:17 +0800,1,Skin Perfecting 2% BHA (Salicylic Acid) Liquid...,13.0,NaN,2016,True,False,pending,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Paula's Choice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,NaN,2016,NaN,1.0,NaN,NaN,NaN,NaN,NaN,13.000000,1.0,13.000000
4,191570000000,isjanelle@gmail.com,paid,2018-05-31 23:54:52 +0800,unfulfilled,NaN,yes,SGD,68.0,0.0,0.0,68.0,NaN,0.0,Free Delivery (2 Working Days),2018-05-31 23:54:52 +0800,1,C1

In [6]:
shopify_data_final.groupby(['Name'])['child_subtotal'].sum()

Name
#1002                112.000000
#1003                 43.000000
#1004                  1.500000
19135641007           37.000000
19135641008            1.500000
19135641009            1.500000
19135641010            1.500000
19135641011            2.000000
19135641012            1.500000
19135641013            1.500000
19135641014            1.500000
19135641015           29.000000
19135641016           29.000000
19135641017           30.500000
19135641019            1.500000
19135641020            1.500000
19135644-1001          1.500000
191567910-1001        48.000000
191567910-1002        76.000000
191567910-1003        36.000000
191567910-1004       157.000000
191567910-1005       133.000000
191567910-1006        59.000000
191567910-1007       245.000000
191567910-1008       160.000000
191567910-1009        48.000000
191567910-1010        61.000000
191567910-1011         3.000000
191567910-1012       181.000000
191567910-1013        89.000000
                      ...      
191